In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# 1. Định nghĩa tập corpus
corpus = [
    "ai is powerful and useful",
    "smart and adaptive system",
    "very smart and useful",
    "learning AI is very hard",
    "ai can be biased",
    "biased and flawed"
]

# 2. Khởi tạo CountVectorizer và tạo ma trận đếm từ (sparse matrix)
vectorizer = CountVectorizer()
X_sparse = vectorizer.fit_transform(corpus)

# 3. In bộ từ vựng (vocabulary)
vocab = dict(sorted(vectorizer.vocabulary_.items()))
print("================== Output =================")
print("Bộ từ vựng xây dựng từ corpus:")
print(vocab)
print("===========================================")

# 4. Chuyển sang mảng dense và in ma trận vector hóa
X = X_sparse.toarray()
print("================== Output =================")
print("Vector đại diện cho các câu trong corpus:")
print(X)
print("===========================================")

================== Output =================
Bộ từ vựng xây dựng từ corpus:
{'adaptive': 0, 'ai': 1, 'and': 2, 'be': 3, 'biased': 4, 'can': 5, 'flawed': 6, 'hard': 7, 'is': 8, 'learning': 9, 'powerful': 10, 'smart': 11, 'system': 12, 'useful': 13, 'very': 14}
================== Output =================
Vector đại diện cho các câu trong corpus:
[[0 1 1 0 0 0 0 0 1 0 1 0 0 1 0]
 [1 0 1 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 0 1 1]
 [0 1 0 0 0 0 0 1 1 1 0 0 0 0 1]
 [0 1 0 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 1 0 0 0 0 0 0 0 0]]


In [2]:
# "ai is powerful and useful"
C1 = np.array([0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0])
# "ai can be biased"
C2 = np.array([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# 4. Chọn các sample cần tính: 1, 2, 3, 5  (chỉ số 0,1,2,4)
sample_indices = [0, 1, 2, 4]

# 5. Tính khoảng cách Euclidean từ C1 tới mỗi sample và in kết quả
print("Khoảng cách Euclidean từ C1 tới các sample 1, 2, 3, 5:")
for idx in sample_indices:
    dist = np.linalg.norm(X[idx] - C1)
    print(f"  - Sample {idx + 1}: {dist:.3f}")

Khoảng cách Euclidean từ C1 tới các sample 1, 2, 3, 5:
  - Sample 1: 0.000
  - Sample 2: 2.646
  - Sample 3: 2.236
  - Sample 5: 2.646


In [3]:
# 5. Tính khoảng cách Euclidean từ C2 tới mỗi sample và in kết quả
print("Khoảng cách Euclidean từ C2 tới các sample 1, 2, 3, 5:")
for idx in sample_indices:
    dist = np.linalg.norm(X[idx] - C2)
    print(f"  - Sample {idx + 1}: {dist:.3f}")

Khoảng cách Euclidean từ C2 tới các sample 1, 2, 3, 5:
  - Sample 1: 2.646
  - Sample 2: 2.828
  - Sample 3: 2.828
  - Sample 5: 0.000


In [5]:
centroids = np.stack([C1, C2], axis=0)  # shape = (2, vocab_size)

# 4. Tính khoảng cách Euclidean tới mỗi tâm cho mọi sample
#    dists has shape (n_samples, 2)
dists = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2)
print("Khoảng cách Euclidean từ các sample tới các tâm:", dists)
print("===========================================")
# 5. Phân cụm: cluster 0 → C1, cluster 1 → C2
cluster_idx = np.argmin(dists, axis=1)  # mảng độ dài 6, giá trị 0 hoặc 1
allocation = ["C1" if ci == 0 else "C2" for ci in cluster_idx]

# 6. Kết quả
print("Vector phân cụm (allocation):")
print(allocation)

Khoảng cách Euclidean từ các sample tới các tâm: [[0.         2.64575131]
 [2.64575131 2.82842712]
 [2.23606798 2.82842712]
 [2.44948974 2.64575131]
 [2.64575131 0.        ]
 [2.44948974 2.23606798]]
Vector phân cụm (allocation):
['C1', 'C1', 'C1', 'C1', 'C2', 'C2']


In [7]:
cluster1 = X[[i for i, a in enumerate(allocation) if a == 'C1']]
cluster2 = X[[i for i, a in enumerate(allocation) if a == 'C2']]

C1_new = np.round(cluster1.mean(axis=0), 2)
C2_new = np.round(cluster2.mean(axis=0), 2)

print("C1 mới:", C1_new)
print("C2 mới:", C2_new)

C1 mới: [0.25 0.5  0.75 0.   0.   0.   0.   0.25 0.5  0.25 0.25 0.5  0.25 0.5
 0.5 ]
C2 mới: [0.  0.5 0.5 0.5 1.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]


In [8]:
# 4. Tính khoảng cách Euclidean từ mỗi sample tới C1_new và C2_new
dist_to_C1 = np.linalg.norm(X - C1_new, axis=1)
dist_to_C2 = np.linalg.norm(X - C2_new, axis=1)

# 5. Phân cụm lại: gần C1_new thì thuộc cụm 1, ngược lại thuộc cụm 2
allocation = []
for d1, d2 in zip(dist_to_C1, dist_to_C2):
    allocation.append('C1' if d1 < d2 else 'C2')

# 6. In kết quả
for idx, cluster in enumerate(allocation, start=1):
    print(f"Sample {idx} thuộc cụm: {cluster}")

# Nếu muốn vector numeric (1 hoặc 2) thay vì 'C1'/'C2':
allocation_numeric = [1 if c == 'C1' else 2 for c in allocation]
print("\nVector phân cụm (numeric):", allocation_numeric)

Sample 1 thuộc cụm: C1
Sample 2 thuộc cụm: C1
Sample 3 thuộc cụm: C1
Sample 4 thuộc cụm: C1
Sample 5 thuộc cụm: C2
Sample 6 thuộc cụm: C2

Vector phân cụm (numeric): [1, 1, 1, 1, 2, 2]
